In [2]:
!python run_clm.py --model_name_or_path "gpt2" --do_train --train_file "../data/wikipedia-10.txt" --max_steps 2000 --per_device_train_batch_size 8 --gradient_accumulation_steps 32 --save_steps 500 --preprocessing_num_workers 1 --counterfactual_augmentation "gender" --seed 0 --output_dir "../results/CDA_checkpoints_seed0_gpt2" --persistent_dir "/home/FYP/on0008an/bias-bench-main" --num_train_epochs 3

06/10/2024 23:22:13 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: True, 16-bits training: False
06/10/2024 23:22:15 - WARNING - datasets.builder - Using custom data configuration default-8f3a313a0598a372
06/10/2024 23:22:15 - WARNING - datasets.builder - Reusing dataset text (/home/FYP/on0008an/.cache/huggingface/datasets/text/default-8f3a313a0598a372/0.0.0/08f6fb1dd2dab0a18ea441c359e1d63794ea8cb53e7863e6edf8fc5655e47ec4)
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 80.48it/s]
06/10/2024 23:22:18 - WARNING - datasets.builder - Using custom data configuration default-8f3a313a0598a372
06/10/2024 23:22:18 - WARNING - datasets.builder - Reusing dataset text (/home/FYP/on0008an/.cache/huggingface/datasets/text/default-8f3a313a0598a372/0.0.0/08f6fb1dd2dab0a18ea441c359e1d63794ea8cb53e7863e6edf8fc5655e47ec4)
06/10/2024 23:22:19 - WARNING - datasets.builder - Using custom data configuration default-8f3a313a0598a372
06/10/2024 23:2

In [ ]:
!python run_clm.py --model_name_or_path "gpt2" --do_train --train_file "../data/wikipedia-10.txt" --max_steps 500 --per_device_train_batch_size 8 --gradient_accumulation_steps 32 --save_steps 500 --preprocessing_num_workers 1 --counterfactual_augmentation "gender" --seed 0 --output_dir "../results/CDA_checkpoints_seed0_gpt2" --persistent_dir "/home/FYP/on0008an/bias-bench-main" --num_train_epochs 3 --overwrite_output_dir